# Limpieza informacion usuario

Este código limpia los datos que tiene Giganav de sus usuarios y los distribuye para ser utilizado para llamadas.

Importo repositorios

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.ticker as mtick
import os
import platform

Establezco directorios

In [2]:
if platform.system()=='Windows': 
    #Directorios Windows
    os.chdir(r"G:\.shortcut-targets-by-id\120bUa35y3FJzvumWahpOx1qS9I6TIrhb\GIGANAV")
    folder=os.getcwd()
    data=folder+'\\'+'Datos'
    facturas=data+'\\'+'Facturas'
    tickets=data+'\\'+'Tickets'
    usuarios=data+'\\'+'Usuarios'
    egresos=data+'\\'+'Egresos'
    transacciones=data+'\\'+'Transacciones'
    analisis=folder+'\\'+'Operacion\\Analisis'
    bases=analisis+'\\'+'Bases_analisis'
    outputs=analisis+'\\'+'Output'
    plots=outputs+'\\'+'Gráficos'
    usuarios_clean=bases+'\\'+'Usuarios'
    transac_clean=bases+'\\'+'Transacciones'
    tickets_clean=bases+'\\'+'Tickets'
    facturas_clean=bases+'\\'+'Facturas'
    egresos_clean=bases+'\\'+'Egresos'
    
if platform.system()=='Darwin':
    #Directorios Mac
    folder='/Users/julian/Library/CloudStorage/GoogleDrive-julian.adiaz@hotmail.com/Mi unidad/Negocios/Familiar/GIGANAV'
    data=folder+'/Datos'
    facturas=data+'/Facturas'
    tickets=data+'/Tickets'
    usuarios=data+'/Usuarios'
    egresos=data+'/Egresos'
    transacciones=data+'/Transacciones'
    analisis=folder+'/Operacion/Analisis'
    bases=analisis+'/Bases_analisis'
    outputs=analisis+'/Outputs'
    plots=outputs+'/Gráficos'
    usuarios_clean=bases+'/Usuarios'
    transac_clean=bases+'/Transacciones'
    tickets_clean=bases+'/Tickets'
    facturas_clean=bases+'/Facturas'
    egresos_clean=bases+'/Egresos'

## OJO debe coincidir con la fecha de descarga de los archivos

In [3]:
fecha=pd.to_datetime('today').strftime('%y_%m_%d')
fecha

'24_09_07'

Importo datos de usuarios

In [4]:
filepath=os.path.join(usuarios,'Lista de Usuarios_'+fecha)
base_usuarios=pd.read_csv(filepath+'.csv')
base_usuarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3847 entries, 0 to 3846
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           0 non-null      float64
 1   Cedula               3845 non-null   object 
 2   Id                   3847 non-null   int64  
 3   Mac                  113 non-null    object 
 4   Nombre               3847 non-null   object 
 5   Dirección Principal  3820 non-null   object 
 6   Deuda actual         1455 non-null   object 
 7   Plan voip            1937 non-null   object 
 8   Plan                 2287 non-null   object 
 9   Ip                   2287 non-null   object 
 10  Tipo estrato         3847 non-null   int64  
 11  Fecha suspendido     3847 non-null   object 
 12  Ultimo pago          3769 non-null   object 
 13  Correo               2913 non-null   object 
 14  Telefono             2875 non-null   object 
 15  Movil                3780 non-null   o

Me quedo con información relevante

In [5]:
depurada_base_usuarios=base_usuarios.loc[:,[
    'Id', 'Nombre', 'Plan voip', 
    'Plan', 'Deuda actual', 'Tipo estrato', 
    'Correo', 'Telefono', 'Movil', 
    'Zona', 'Total cobrar']]
depurada_base_usuarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3847 entries, 0 to 3846
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            3847 non-null   int64 
 1   Nombre        3847 non-null   object
 2   Plan voip     1937 non-null   object
 3   Plan          2287 non-null   object
 4   Deuda actual  1455 non-null   object
 5   Tipo estrato  3847 non-null   int64 
 6   Correo        2913 non-null   object
 7   Telefono      2875 non-null   object
 8   Movil         3780 non-null   object
 9   Zona          3820 non-null   object
 10  Total cobrar  3847 non-null   object
dtypes: int64(2), object(9)
memory usage: 330.7+ KB


In [6]:
depurada_base_usuarios.head()

,Id,Nombre,Plan voip,Plan,Deuda actual,Tipo estrato,Correo,Telefono,Movil,Zona,Total cobrar
0,3376,Prueba Prueba RETIRADO,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,"COL$ 0,00"
1,3378,PRUEBA GIGANAV RETIRADO,NaN,NaN,NaN,1,NaN,NaN,NaN,VILLA FANNY,"COL$ 0,00"
2,1374,RODRIGO SILVA SEPULVEDA SUSPENDIDO,NaN,GIGABASIC RURAL,"1 COL$ 122.000,00",1,silva_rodrigo_25@hotmail.com,3206671131,3153598524,CGTO MINAS,"COL$ 120.000,00"
3,2450,PUNTO GRATIS ACTIVO,NaN,30X30X70 GPON3,NaN,1,NaN,NaN,NaN,NaN,"COL$ 0,00"
4,788,JHON FREDY FONSECA CARRANZA RETIRADO,NaN,NaN,"3 COL$ 182.000,00",1,jhonfredyfonsecacarranza@gmail.com,3162662078,3187613025,CGTO LA LLANA,"COL$ 0,00"


Creo categorias de usuarios y limpio nombre

In [7]:
categorias_usuarios=['activo', 'retirado', 'suspendido']

for j, i in enumerate(categorias_usuarios):
    #Creo estado del usuario
    depurada_base_usuarios.loc[
        depurada_base_usuarios['Nombre'].str.contains(i, case=False), 
        'estado']=categorias_usuarios[j]
    
#Creo nombre corregido
depurada_base_usuarios.loc[:, 'nombre']=depurada_base_usuarios[
    'Nombre'].str.replace('activo', '', case=False)
depurada_base_usuarios.loc[:, 'nombre']=depurada_base_usuarios[
    'nombre'].str.replace('retirado', '', case=False)
depurada_base_usuarios.loc[:, 'nombre']=depurada_base_usuarios[
    'nombre'].str.replace('suspendido', '', case=False)

Creo variable de TV

In [8]:
#Lleno Nan
depurada_base_usuarios.loc[:,'Plan voip'].fillna('', inplace=True)

#Creo variable
depurada_base_usuarios.loc[:, 'TV']=depurada_base_usuarios[
    'Plan voip'].str.contains('recaudo a terceros', case=False)*1

depurada_base_usuarios['TV'].value_counts()

1    1937
0    1910
Name: TV, dtype: int64

Creo variable de megas

In [9]:
#Creo megas que maneja la empresa
megas=[10, 15,25, 30, 40, 50, 60, 100, 150]

#Limpio Nan de variable plan
depurada_base_usuarios.loc[:, 'Plan'].fillna('', inplace=True)

#Creo variable megas
depurada_base_usuarios.loc[:,'megas']=np.select([
    depurada_base_usuarios['Plan'].str.contains('10X10', case=False), 
    depurada_base_usuarios['Plan'].str.contains('15X15', case=False),  
    depurada_base_usuarios['Plan'].str.contains('25X25', case=False), 
    depurada_base_usuarios['Plan'].str.contains('30X30', case=False), 
    depurada_base_usuarios['Plan'].str.contains('40X40', case=False), 
    depurada_base_usuarios['Plan'].str.contains('50X50', case=False), 
    depurada_base_usuarios['Plan'].str.contains('60X60', case=False), 
    depurada_base_usuarios['Plan'].str.contains('100X100|100X30', case=False), 
    depurada_base_usuarios['Plan'].str.contains('150X150', case=False),], 
    choicelist=megas)
#Limpio vacíos
depurada_base_usuarios.loc[
    depurada_base_usuarios['megas']=='0','megas']=0

depurada_base_usuarios['megas'].value_counts().sort_values(ascending=False)

0      1750
50     1115
10      296
15      174
100     173
60      118
30       91
25       78
40       49
150       3
Name: megas, dtype: int64

Visualizo número de usuarios por segmento de la empresa

In [10]:
mask_megas=depurada_base_usuarios['megas']!=0

pd.pivot_table(
    depurada_base_usuarios.loc[mask_megas], index='megas', columns='TV', values='Id', aggfunc='nunique')

TV,0,1
megas,,
10,98,198
15,34,140
25,22,56
30,34,57
40,12,37
50,404,711
60,45,73
100,85,88
150,2,1


Creo variable radio

In [11]:
depurada_base_usuarios.loc[:, 'radio']=depurada_base_usuarios[
    'Plan'].str.contains('giga', case=False)*1

depurada_base_usuarios['radio'].value_counts()

0    3657
1     190
Name: radio, dtype: int64

In [12]:
#Creo total a cobrar
depurada_base_usuarios.loc[:,'tarifa']=depurada_base_usuarios['Total cobrar'].str.strip('COL$ ')
depurada_base_usuarios.loc[:,'tarifa']=depurada_base_usuarios['tarifa'].str.replace('.','')
depurada_base_usuarios.loc[:,'tarifa']=depurada_base_usuarios['tarifa'].str.replace(',','.')
depurada_base_usuarios.loc[:,'tarifa']=depurada_base_usuarios['tarifa'].astype('float')
depurada_base_usuarios.loc[
    depurada_base_usuarios['TV']==1,'tarifa']=depurada_base_usuarios['tarifa']+25000

<ipython-input-12-3d4fe6ac948c>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  depurada_base_usuarios.loc[:,'tarifa']=depurada_base_usuarios['tarifa'].str.replace('.','')


In [13]:
depurada_base_usuarios.loc[depurada_base_usuarios['estado']=='activo']

,Id,Nombre,Plan voip,Plan,Deuda actual,Tipo estrato,Correo,Telefono,Movil,Zona,Total cobrar,estado,nombre,TV,megas,radio,tarifa
3,2450,PUNTO GRATIS ACTIVO,,30X30X70 GPON3,NaN,1,NaN,NaN,NaN,NaN,"COL$ 0,00",activo,PUNTO GRATIS,0,30,0,0.0
6,1722,MARIA ALEJANDRA RESTREPO MORALES ACTIVO,RECAUDO A TERCEROS,60X60X60 GPON 1,NaN,1,Marymkery124@gmail.com,3128227880,3105378643,LOS PINOS,"COL$ 55.000,00",activo,MARIA ALEJANDRA RESTREPO MORALES,1,60,0,80000.0
7,3806,2-MARIA ALEJANDRA RESTREPO MORALES ACTIVO,RECAUDO A TERCEROS,50X50X150 GPON3,NaN,1,marymkery124@gmail.com,3123140321,3105378643,1º DE MAYO,"COL$ 45.000,00",activo,2-MARIA ALEJANDRA RESTREPO MORALES,1,50,0,70000.0
10,3399,YULUAN STEVEN ORTIZ GIRALDO ACTIVO,RECAUDO A TERCEROS,50X50X150 GPON1,NaN,1,ruizsernasiomara@gmail.com,3188003166,3162122229,VILLA SAN BERNARDO,"COL$ 45.000,00",activo,YULUAN STEVEN ORTIZ GIRALDO,1,50,0,70000.0
13,4079,ANGIE CAROLINA FLOREZ AGUDELO ACTIVO,RECAUDO A TERCEROS,50X50X150 GPON3,NaN,1,florezangie1820@gmail.com,3188291559,3167024209,EL OASIS,"COL$ 45.000,00",activo,ANGIE CAROLINA FLOREZ AGUDELO,1,50,0,70000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3836,2380,2-FREDY PEREZ MORA ACTIVO,,50X50X150 GPON2,NaN,1,fredyperezmora1982@gmail.com,3188868005,3172972735,CGTO LA PEDREGOSA,"COL$ 50.000,00",activo,2-FREDY PEREZ MORA,0,50,0,50000.0
3839,2638,ANDERSON SALAS AYALA ACTIVO,RECAUDO A TERCEROS,25X25X65 GPON3,NaN,1,andersonsalas04@gmail.com,3176908526,3188734677,BETANCURT,"COL$ 48.000,00",activo,ANDERSON SALAS AYALA,1,25,0,73000.0
3840,3879,JOSE LUIS PEREZ MONTERO ACTIVO,,60X60X60 GPON 3,NaN,1,jperez.jlpm24@gmail.com,3122140668,3107291306,SAN RAFAEL,"COL$ 60.000,00",activo,JOSE LUIS PEREZ MONTERO,0,60,0,60000.0
3842,21,ODAIR MORA VALBUENA ACTIVO,,10X10X50 GPON2,NaN,2,odairmoravalbuena@gmail.com,NaN,3152180728,NUEVO HORIZONTE,"COL$ 50.000,00",activo,ODAIR MORA VALBUENA,0,10,0,50000.0


In [14]:
variables=['Id', 'nombre', 'estado', 
           'radio', 'Tipo estrato', 'megas', 
           'TV', 'Telefono', 'Movil', 
           'Correo', 'Zona', 'tarifa', 
           'Deuda actual']
base_final_contactos=depurada_base_usuarios.loc[:,variables]
base_final_contactos

,Id,nombre,estado,radio,Tipo estrato,megas,TV,Telefono,Movil,Correo,Zona,tarifa,Deuda actual
0,3376,Prueba Prueba,retirado,0,1,0,0,NaN,NaN,NaN,NaN,0.0,NaN
1,3378,PRUEBA GIGANAV,retirado,0,1,0,0,NaN,NaN,NaN,VILLA FANNY,0.0,NaN
2,1374,RODRIGO SILVA SEPULVEDA,suspendido,1,1,0,0,3206671131,3153598524,silva_rodrigo_25@hotmail.com,CGTO MINAS,120000.0,"1 COL$ 122.000,00"
3,2450,PUNTO GRATIS,activo,0,1,30,0,NaN,NaN,NaN,NaN,0.0,NaN
4,788,JHON FREDY FONSECA CARRANZA,retirado,0,1,0,0,3162662078,3187613025,jhonfredyfonsecacarranza@gmail.com,CGTO LA LLANA,0.0,"3 COL$ 182.000,00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3842,21,ODAIR MORA VALBUENA,activo,0,2,10,0,NaN,3152180728,odairmoravalbuena@gmail.com,NUEVO HORIZONTE,50000.0,NaN
3843,2875,ALIRIO DE JESUS MONSALVE OSPINA,retirado,0,1,0,0,3126429826,3213262547,aliriooodejesusmonsalve1805@gmail.com,CGTO LA PEDREGOSA,0.0,"1 COL$ 25.000,00"
3844,1038,JORGE ALBERTO MORENO,suspendido,0,1,50,0,3142082576,3188925978,NaN,CGTO LA PEDREGOSA,52000.0,"1 COL$ 54.000,00"
3845,3265,JESUS MARIA HERNANDEZ HENAO,retirado,0,1,0,1,3145291879,3208370958,jesusmariahernan@gmail.com,CGTO LA PEDREGOSA,25000.0,NaN


In [15]:
#Leo base Hubspot y agrego record ID para actualizar contactos existentes

In [16]:
filepath=os.path.join(usuarios,'Usuarios_export_Hubspot_'+fecha)
base_hubspot=pd.read_csv(filepath+'.csv')
base_hubspot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3909 entries, 0 to 3908
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID de registro           3909 non-null   int64  
 1   Nombre                   3903 non-null   object 
 2   Apellidos                14 non-null     object 
 3   estado suscripción       3880 non-null   object 
 4   Tarifa                   3880 non-null   float64
 5   Megas                    3880 non-null   float64
 6   TV                       3880 non-null   float64
 7   Telefono                 2932 non-null   object 
 8   Movil                    3814 non-null   object 
 9   Correo de miembro        2910 non-null   object 
 10  Última actividad         292 non-null    object 
 11  ID_mikrowisp             3880 non-null   float64
 12  Zona                     3855 non-null   object 
 13  Etapa del ciclo de vida  3909 non-null   object 
 14  meses_suspendido        

In [17]:
#Cambio nombre y hago merge para pegar solo el Record ID de Hubspot
#Elimino duplicados y me quedo con el record de hubspot mas reciente
base_hubspot.rename(columns={
    'ID_mikrowisp': 'Id'}, inplace=True)

base_hubspot=base_hubspot.groupby(
        'Id').agg(
        id_hubspot=(
        'ID de registro', 'max')).reset_index()

base_hubspot.rename(columns={
    'id_hubspot': 'ID de registro'}, inplace=True)

#Hago merge
base_records_hubspot=base_final_contactos.merge(base_hubspot.loc[
    :,['Id', 'ID de registro']], on='Id', how='left')

base_records_hubspot.loc[
    :, 'ID de registro']=base_records_hubspot[
    'ID de registro'].astype('Int64')

base_records_hubspot.head()

,Id,nombre,estado,radio,Tipo estrato,megas,TV,Telefono,Movil,Correo,Zona,tarifa,Deuda actual,ID de registro
0,3376,Prueba Prueba,retirado,0,1,0,0,NaN,NaN,NaN,NaN,0.0,NaN,21911
1,3378,PRUEBA GIGANAV,retirado,0,1,0,0,NaN,NaN,NaN,VILLA FANNY,0.0,NaN,20907
2,1374,RODRIGO SILVA SEPULVEDA,suspendido,1,1,0,0,3206671131,3153598524,silva_rodrigo_25@hotmail.com,CGTO MINAS,120000.0,"1 COL$ 122.000,00",25003
3,2450,PUNTO GRATIS,activo,0,1,30,0,NaN,NaN,NaN,NaN,0.0,NaN,35701
4,788,JHON FREDY FONSECA CARRANZA,retirado,0,1,0,0,3162662078,3187613025,jhonfredyfonsecacarranza@gmail.com,CGTO LA LLANA,0.0,"3 COL$ 182.000,00",25405


In [18]:
base_records_hubspot.loc[
    :,'meses_suspendido']= base_records_hubspot.loc[
    base_records_hubspot['estado']=='suspendido',
    'Deuda actual'].str.split(
    ' ', expand=True, n=1).iloc[:,0].fillna(0).astype(
    'int')

base_records_hubspot.loc[
    :,'deuda']= base_records_hubspot.loc[
    base_records_hubspot['estado']=='suspendido',
    'Deuda actual'].str.split(
    ' ', expand=True, n=1).iloc[:,1].str.strip(
    'COL$').str.replace(
    '.','').str.replace(
    ',00','').str.lstrip().str.replace(',','.').fillna(0).astype('float')



<ipython-input-18-1a1593a2fa95>:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  :,'deuda']= base_records_hubspot.loc[


In [19]:
print('El número de usuarios suspendidos al '+fecha+' es '+str(base_records_hubspot.loc[
    base_records_hubspot['estado']=='suspendido', 'Id'].groupby(level=0).nunique().sum()))

El número de usuarios suspendidos al 24_09_07 es 462


Exporto base para hubspot

In [20]:
filepath=os.path.join(usuarios_clean,'Base_usuarios_hub_'+fecha)
base_records_hubspot.to_csv(filepath+'.csv', index=False)

##### 